## Code for making the cell volume from cells detected and registered to Princeton Mouse Atlas

In [1]:
import numpy as np
import tifffile as tif
import cv2
import matplotlib.pyplot as plt
from skimage.morphology import ball

In [2]:
#first, make a map of cells
pth = "/jukebox/wang/Jess/lightsheet_output/201904_ymaze_cfos/processed/an31/clearmap_cluster_output/cells.npy"
converted_points = np.load(pth)

xyz = np.asarray([(int(xx[0]), int(xx[1]), int(xx[2])) for xx in converted_points]) #cells are counted in horizontal volumes

In [3]:
#init empty vol 
cell_map = np.zeros((637,2560,2160)).astype('uint16')

In [4]:
#fill map
for x,y,z in xyz:
    try:
        cell_map[z-1:z+1,y,x] = 1 #z dilation
    except Exception as e:
        print(e)

In [7]:
#apply x y dilation
r = 2
selem = ball(r)[int(r/2)]
# cell_map = cell_map.astype("uint8")
cell_map = np.asarray([cv2.dilate(cell_map[i], selem, iterations = 1) for i in range(cell_map.shape[0])])

In [8]:
# Now create the cloudvolume for this tif file
import os
import numpy as np

from concurrent.futures import ProcessPoolExecutor

from taskqueue import LocalTaskQueue
import igneous.task_creation as tc


from cloudvolume import CloudVolume
from cloudvolume.lib import mkdir, touch
from PIL import Image

In [14]:
home_dir = '/home/ahoag/ngdemo'
progress_dir = mkdir(home_dir + '/progress_201904_ymaze_cfos_rawcells_an31_dilate/')
def make_info_file():
    info = CloudVolume.create_new_info(
        num_channels = 1,
        layer_type = 'segmentation', # 'image' or 'segmentation'
        data_type = 'uint16', # can pick any popular uint
        encoding = 'raw', # other options: 'jpeg', 'compressed_segmentation' (req. uint32 or uint64)
        resolution = [ 5000, 5000, 10000 ], # X,Y,Z values in nanometers, 20 microns in each dim
        voxel_offset = [ 0, 0, 0 ], # values X,Y,Z values in voxels
        chunk_size = [ 1024, 1024, 1 ], # rechunk of image X,Y,Z in voxels
        volume_size = [2160,2560,637], # X,Y,Z size in voxels
    )

    # If you're using amazon or the local file system, you can replace 'gs' with 's3' or 'file'
    vol = CloudVolume('file:///home/ahoag/ngdemo/demo_bucket/201904_ymaze_cfos/rawcells_an31_dilate', info=info)
    vol.provenance.description = "Show cells registered to the Princeton Mouse atlas"
    vol.provenance.owners = ['ahoag@princeton.edu'] # list of contact email addresses

    vol.commit_info() # generates gs://bucket/dataset/layer/info json file
    vol.commit_provenance() # generates gs://bucket/dataset/layer/provenance json file
    return vol

In [15]:
def process_slice(z):
    print('Processing slice z=',z)
    
    array = cell_map[z].reshape((1,y_dim,x_dim)).T

    cloud_vol[:,:, z] = array
    touch(os.path.join(progress_dir, str(z)))
    print("success")

In [16]:
cloud_vol = make_info_file()
""" tifffile is already loaded """
# image = np.array(tifffile.imread(brainvolume_file),dtype=np.uint16, order='F') # F stands for fortran order
z_dim,y_dim,x_dim = cell_map.shape

done_files = set([ int(z) for z in os.listdir(progress_dir) ])
all_files = set(range(cloud_vol.bounds.minpt.z, cloud_vol.bounds.maxpt.z)) # 1 indexed 

to_upload = [ int(z) for z in list(all_files.difference(done_files)) ]
to_upload.sort()
# print("Remaining slices to upload are:",to_upload)

with ProcessPoolExecutor(max_workers=8) as executor:
    executor.map(process_slice, to_upload)

Processing slice z= 0
Processing slice z= 1
Processing slice z= 6
Processing slice z= 5
Processing slice z= 4
Processing slice z= 3
Processing slice z= 2
Processing slice z= 7


Uploading:   0%|          | 0/1 [00:00<?, ?it/s]


Uploading:   0%|          | 0/1 [00:00<?, ?it/s]




Uploading:   0%|          | 0/1 [00:00<?, ?it/s]






Uploading:   0%|          | 0/1 [00:00<?, ?it/s]



Uploading:   0%|          | 0/1 [00:00<?, ?it/s]


Uploading:   0%|          | 0/1 [00:00<?, ?it/s]

Uploading:   0%|          | 0/1 [00:00<?, ?it/s]




Uploading:   0%|          | 0/1 [00:00<?, ?it/s]



Uploading:   0%|          | 0/1 [00:00<?, ?it/s]




Uploading:   0%|          | 0/1 [00:00<?, ?it/s]

Uploading:   0%|          | 0/1 [00:00<?, ?it/s]

Uploading:   0%|          | 0/1 [00:00<?, ?it/s]

Uploading:   0%|          | 0/1 [00:00<?, ?it/s]


Uploading:   0%|          | 0/1 [00:00<?, ?it/s]



Uploading:   0%|          | 0/1 [00:00<?, ?it/s]


Uploading:   0%|          | 0/1 [00:00<?, ?it/s]



Uploading:   0%|          | 0/1 [00:00<?, ?it/s]



Uploading:   0%|          | 0/1 [00:00<?, ?it/s]

Uploading:   0%|          | 0/1 [00:00<?, ?it/s]


Uploading:   0%|     

Uploading: 100%|██████████| 1/1 [00:00<00:00,  5.96it/s]

Uploading: 100%|██████████| 1/1 [00:00<00:00,  6.07it/s]


Uploading: 100%|██████████| 1/1 [00:00<00:00,  6.20it/s]




Uploading: 100%|██████████| 1/1 [00:00<00:00,  6.05it/s]


Uploading: 100%|██████████| 1/1 [00:00<00:00,  6.28it/s]





Uploading: 100%|██████████| 1/1 [00:00<00:00,  6.02it/s]




Uploading: 100%|██████████| 1/1 [00:00<00:00,  6.07it/s]




Uploading: 100%|██████████| 1/1 [00:00<00:00,  6.20it/s]



Uploading: 100%|██████████| 1/1 [00:00<00:00,  6.03it/s]



Uploading: 100%|██████████| 1/1 [00:00<00:00,  6.04it/s]



Uploading: 100%|██████████| 1/1 [00:00<00:00,  6.16it/s]

Uploading: 100%|██████████| 1/1 [00:00<00:00,  6.26it/s]




Uploading: 100%|██████████| 1/1 [00:00<00:00,  5.91it/s]

Uploading: 100%|██████████| 1/1 [00:00<00:00,  5.89it/s]




Uploading: 100%|██████████| 1/1 [00:00<00:00,  6.07it/s]


Uploading: 100%|██████████| 1/1 [00:00<00:00,  6.11it/s]



Uploading: 100%|██████████| 1/1 [00:00<00:

success
Processing slice z= 8
success
success
Processing slice z= 9
Processing slice z= 10
success
success
success
Processing slice z= 11
Processing slice z= 13
success
Processing slice z= 12
success
Processing slice z= 14
Processing slice z= 15



Uploading:   0%|          | 0/9 [00:00<?, ?it/s]








Uploading:   0%|          | 0/1 [00:00<?, ?it/s]

Uploading:   0%|          | 0/1 [00:00<?, ?it/s]


Uploading:   0%|          | 0/1 [00:00<?, ?it/s]




Uploading:   0%|          | 0/1 [00:00<?, ?it/s]





Uploading:   0%|          | 0/1 [00:00<?, ?it/s]






Uploading:   0%|          | 0/1 [00:00<?, ?it/s]







Uploading:   0%|          | 0/1 [00:00<?, ?it/s]


Uploading:   0%|          | 0/1 [00:00<?, ?it/s]






Uploading:   0%|          | 0/9 [00:00<?, ?it/s]

Uploading:   0%|          | 0/1 [00:00<?, ?it/s]


Uploading:   0%|          | 0/1 [00:00<?, ?it/s]



Uploading:   0%|          | 0/1 [00:00<?, ?it/s]

Uploading:   0%|          | 0/1 [00:00<?, ?it/s]







Uploading:   0%|          | 0/1 [00:00<?, ?it/s]



Uploading:   0%|          | 0/1 [00:00<?, ?it/s]

Uploading:   0%|          | 0/9 [00:00<?, ?it/s]


Uploading:   0%|          | 0/1 [00:00<?, ?it/s]



Uploading:   0%|          | 0/1 [00:00<?, ?it/s]






...

Uploading: 100%|██████████| 1/1 [00:00<00:00,  6.12it/s]








Uploading: 100%|██████████| 1/1 [00:00<00:00,  5.86it/s]



Uploading: 100%|██████████| 1/1 [00:00<00:00,  5.87it/s]





Uploading: 100%|██████████| 1/1 [00:00<00:00,  5.60it/s]





Uploading: 100%|██████████| 1/1 [00:00<00:00,  5.79it/s]






Uploading: 100%|██████████| 1/1 [00:00<00:00,  5.75it/s]


Uploading: 100%|██████████| 1/1 [00:00<00:00,  5.93it/s]








Uploading: 100%|██████████| 1/1 [00:00<00:00,  5.61it/s]





Uploading: 100%|██████████| 1/1 [00:00<00:00,  5.84it/s]




Uploading: 100%|██████████| 1/1 [00:00<00:00,  5.69it/s]







Uploading: 100%|██████████| 1/1 [00:00<00:00,  5.42it/s]



Uploading: 100%|██████████| 1/1 [00:00<00:00,  5.87it/s]




Uploading: 100%|██████████| 1/1 [00:00<00:00,  5.84it/s]






Uploading: 100%|██████████| 1/1 [00:00<00:00,  5.47it/s]








Uploading: 100%|██████████| 1/1 [00:00<00:00,  5.54it/s]


Uploading: 100%|██████████| 1/1 [00:00<00:00,  5.87it/s]






Upload

success
Processing slice z= 617
success
Processing slice z= 618
success
success
Processing slice z= 619
Processing slice z= 620
success
Processing slice z= 621
success
success
success
Processing slice z= 622
Processing slice z= 624
Processing slice z= 623



Uploading:   0%|          | 0/1 [00:00<?, ?it/s]

Uploading:   0%|          | 0/1 [00:00<?, ?it/s]


Uploading:   0%|          | 0/1 [00:00<?, ?it/s]

Uploading:   0%|          | 0/1 [00:00<?, ?it/s]






Uploading:   0%|          | 0/9 [00:00<?, ?it/s]


Uploading:   0%|          | 0/1 [00:00<?, ?it/s]

Uploading:   0%|          | 0/1 [00:00<?, ?it/s]

Uploading:   0%|          | 0/1 [00:00<?, ?it/s]








Uploading:   0%|          | 0/1 [00:00<?, ?it/s]

Uploading:   0%|          | 0/1 [00:00<?, ?it/s]

Uploading:   0%|          | 0/9 [00:00<?, ?it/s]


Uploading:   0%|          | 0/1 [00:00<?, ?it/s]








Uploading:   0%|          | 0/1 [00:00<?, ?it/s]

Uploading:   0%|          | 0/1 [00:00<?, ?it/s]


Uploading:   0%|          | 0/1 [00:00<?, ?it/s]


Uploading:   0%|          | 0/1 [00:00<?, ?it/s]


Uploading:   0%|          | 0/1 [00:00<?, ?it/s]



Uploading:   0%|          | 0/1 [00:00<?, ?it/s]


Uploading:   0%|          | 0/1 [00:00<?, ?it/s]



Uploading:   0%|   

Uploading: 100%|██████████| 1/1 [00:00<00:00, 14.15it/s]






Uploading: 100%|██████████| 1/1 [00:00<00:00, 14.48it/s]





Uploading: 100%|██████████| 1/1 [00:00<00:00, 16.10it/s]







Uploading: 100%|██████████| 1/1 [00:00<00:00, 13.78it/s]

Uploading: 100%|██████████| 1/1 [00:00<00:00, 13.75it/s]


Uploading: 100%|██████████| 1/1 [00:00<00:00, 14.99it/s]





Uploading: 100%|██████████| 1/1 [00:00<00:00, 15.93it/s]




Uploading: 100%|██████████| 1/1 [00:00<00:00, 14.75it/s]



Uploading: 100%|██████████| 1/1 [00:00<00:00, 13.71it/s]



Uploading: 100%|██████████| 1/1 [00:00<00:00, 16.69it/s]






Uploading: 100%|██████████| 1/1 [00:00<00:00, 14.21it/s]

Uploading: 100%|██████████| 1/1 [00:00<00:00, 14.06it/s]


Uploading: 100%|██████████| 1/1 [00:00<00:00,  6.54it/s]









Uploading: 100%|██████████| 1/1 [00:00<00:00, 15.45it/s]


Uploading: 100%|██████████| 1/1 [00:00<00:00,  6.77it/s]





Uploading: 100%|██████████| 1/1 [00:00<00:00, 13.24it/s]







Uploading: 100%|████

success
success
success
success


In [ ]:
# make the igenous mesh to display in 3D
def make_mesh():
    # Mesh on 8 cores, use True to use all cores
    cloudpath = cloud_vol.cloudpath
    with LocalTaskQueue(parallel=8) as tq:
      tasks = tc.create_meshing_tasks(cloudpath, mip=0, shape=(256, 256, 256))
      tq.insert_all(tasks)
      tasks = tc.create_mesh_manifest_tasks(cloudpath)
      tq.insert_all(tasks)
    print("Done!")

In [ ]:
make_mesh()

In [10]:
# Start a viewer and see what it looks like
import neuroglancer
from collections import OrderedDict
neuroglancer.set_static_content_source(url='https://neuromancer-seung-import.appspot.com')

In [ ]:
# First need to host the cloudvolume -- did that in the terminal at port 1337

In [19]:
viewer = neuroglancer.Viewer()
# This volume handle can be used to notify the viewer that the data has changed.

with viewer.txn() as s:
    s.layers['an25'] = neuroglancer.SegmentationLayer(source='precomputed://http://localhost:1337'
    )
    s.layers['an2'] = neuroglancer.SegmentationLayer(source='precomputed://http://localhost:1338'
    )
    s.layers['atlas'] = neuroglancer.SegmentationLayer(source='precomputed://http://localhost:1339'
    )
with viewer.txn() as s:
    s.layout = neuroglancer.row_layout(
        [neuroglancer.LayerGroupViewer(layers=['an25','atlas']),
         neuroglancer.LayerGroupViewer(layers=['an2','atlas']),
         neuroglancer.LayerGroupViewer(layers=['an2','atlas'])])
#     s.layers[0]._json_data['segments']=unique_segments
print(viewer)

http://127.0.0.1:39249/v/ba85b383e85cba7e38ab4d771dacebbaae4eaffa/


In [ ]:
with viewer.txn() as s:
    s.layers['atlas'] = neuroglancer.SegmentationLayer(source='precomputed://http://localhost:1338'
    )

I loaded the allen atlas on there as well and it looks like the atlas is offset in z by a few hundred planes. Let's look at the allen atlas info file to see what is going on

In [ ]:
allen_vol = CloudVolume('file:///home/ahoag/ngdemo/demo_bucket/atlas/allenatlas/')

In [ ]:
allen_vol.info

Yeah, looks like the x and z dimensions are swapped. Just remake the allen atlas with the correct dimensions and it should be fine. 

In [ ]:
ss = neuroglancer.ScreenshotSaver(viewer, '/home/ahoag/ngdemo/screenshots/')
ss.capture()  